### Exercise 4 Neural Network Learning

#### Parameters

In [7]:
input_layer_size  = 400;  % 20x20 Input Images of Digits
hidden_layer_size = 25;   % 25 hidden units
num_labels = 10;          % 10 labels, from 1 to 10 (we have mapped "0" to label 10)
fprintf('input_layer_size = %d, hidden_layer_size = %d, num_labels = %d\n',input_layer_size, hidden_layer_size, num_labels);

input_layer_size = 400, hidden_layer_size = 25, num_labels = 10


#### Part 1: Loading and Visualizing Data

In [8]:
load('ex4data1.mat');
m = size(X, 1);
fprintf('m = size(X, 1) = %d %d\n', size(X, 1));

% Randomly select 100 data points to display
%sel = randperm(size(X, 1));
%sel = sel(1:100);

graphics_toolkit ('gnuplot');
%displayData(X(sel, :));

m = size(X, 1) = 5000 

#### Part 2: Loading Parameters

In [9]:
% Load the weights into variables Theta1 and Theta2
load('ex4weights.mat');

% Unroll parameters 
nn_params = [Theta1(:) ; Theta2(:)];

#### Part 3: Compute Cost (Feedforward)

In [10]:
fprintf('m = size(X, 1) = %d\n', size(X, 1))

m = size(X, 1) = 5000


In [11]:
% Weight regularization parameter (we set this to 0 here).
lambda = 1;

In [7]:
fprintf('size Theta1 = %d %d; size Theta2 = %d %d\n', size(Theta1), size(Theta2))

size Theta1 = 25 401; size Theta2 = 10 26


### Brack propagation

In [ ]:
Theta1 = reshape(nn_params(1:hidden_layer_size * (input_layer_size + 1)), ...
                 hidden_layer_size, (input_layer_size + 1));
Theta2 = reshape(nn_params((1 + (hidden_layer_size * (input_layer_size + 1))):end), ...
                 num_labels, (hidden_layer_size + 1));

m = size(X, 1);
         
J = 0;
Theta1_grad = zeros(size(Theta1)); % 25 * 401
Theta2_grad = zeros(size(Theta2)); % 10 * 26

X_add1 = [ones(m,1) X];

%%% old
activation2 = sigmoid(X_add1 * Theta1');  %  Old: 5000 * 401 * 401 * 25
activation2 = [ones(m,1) activation2]; %  5000 * 26
Htheta = sigmoid(activation2 * Theta2'); % 5000 * 10, suit Htheta(i,k) 

eyebase = eye(num_labels);
y_matrix = eyebase(y,:); 


for i = 1 : m
    for k = 1 : num_labels
        J = J + ( - y_matrix(i,k) * log(Htheta(i,k)) - (1 - y_matrix(i,k)) * log(1 - Htheta(i,k)));
    end
end

J = 1/m * J;  

% -------------------------------------------------------------

%%% ex4 1.4 Regularized Cost Function

regTerm_Theta1 = 0;
regTerm_Theta2 = 0;

for j = 1 : hidden_layer_size
    for k = 1 : input_layer_size
        regTerm_Theta1 = regTerm_Theta1 + Theta1(j,k+1)^2;
    end
end

for j = 1 : num_labels
    for k = 1 : hidden_layer_size
        regTerm_Theta2 = regTerm_Theta2 + Theta2(j,k+1)^2;
    end
end

J = J + lambda/(2*m) * (regTerm_Theta1 + regTerm_Theta2);

% Part 2: Implement the backpropagation algorithm to compute the gradients
%         Theta1_grad and Theta2_grad. You should return the partial derivatives of
%         the cost function with respect to Theta1 and Theta2 in Theta1_grad and
%         Theta2_grad, respectively. After implementing Part 2, you can check
%         that your implementation is correct by running checkNNGradients
%
%         Note: The vector y passed into the function is a vector of labels
%               containing values from 1..K. You need to map this vector into a 
%               binary vector of 1's and 0's to be used with the neural network
%               cost function.
%
%         Hint: We recommend implementing backpropagation using a for-loop
%               over the training examples if you are implementing it for the 
%               first time.


delta_3 = zeros(num_labels, 1); % 10 * 1

BigDelta_2 = zeros(size(Theta2_grad));
BigDelta_1 = zeros(size(Theta1_grad));

% for loop for step 1 to 4
for t = 1 : m
    for k = 1 : num_labels

        %%% Lab step 1:
        % denote a^{(l)} (the lth layer) as a_l. (al normally used as elemental 'fen liang') 

        a_1 = X(t,:)'; %  column vector
        a_1 = [1;a_1]; % 401 * 1
        z_2 = Theta1 * a_1; % 25 * 401 * 401 * 1 = 25 * 1
        a_2 = sigmoid(z_2); 
        a_2 = [1;a_2]; % 26 * 1
        z_3 = Theta2 * a_2; % 10 * 26 * 26 * 1 = 10 * 1
        a_3 = sigmoid(z_3); % 10 * 1


        %%% Lab step 2:               
        delta_3(k) = a_3(k) - y_matrix(t,k); 
    end
    
        %%% Lab step 3:
        % delta_2 = zeros(hidden_layer_size,1);        
        %% 1st try, a little different than lab note p.9 in terms of notation (ignore bias unit in Theta instead of delta)      
        delta_2 = Theta2(:,2:end)' * delta_3 .* sigmoidGradient(z_2);  % 25 * 10 * 10 * 1 .* 25 * 1
        % ignoring the Theta2 bias units

  
        %%% Lab step 4:

        BigDelta_2 = BigDelta_2 + delta_3 * a_2';  % 10 * 1 * 1 * 26 = 10 * 26  num_labels * hidden_layer_size
        BigDelta_1 = BigDelta_1 + delta_2 * a_1';  % 25 * 1 * 1 * 401 = 25 * 401  hidden_layer_size * input_layer_size
    
end

fprintf('size(BigDelta_2) %d %d, size(BigDelta_2) %d %d',size(BigDelta_2), size(BigDelta_1))
% -------------------------------------------------------------


%%% Lab step 5:

Theta1_grad = BigDelta_1 / m;

Theta2_grad = BigDelta_2 / m;
